## Домашнее задание <a class='anchor' id='hw'>

1. *Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.


3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).


4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

5. *Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

In [3]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [4]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

In [5]:
def calc_logloss(y, y_pred):
    y = y.astype(np.float64)
    y_pred = y_pred.astype(np.float64)
    for i in range(len(y)):
        if y[i] == 0:
            y[i] += 1e-10
        if y[i] == 1:
            y[i] -= 1e-10
    for i in range(len(y_pred)):
        if y_pred[i] == 0:
            y_pred[i] += 1e-10
        if y_pred[i] == 1:
            y_pred[i] -= 1e-10
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [6]:
# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([0.8, 0.1])
calc_logloss(y1, y_pred1)

0.16425203366519395

In [7]:
# Плохой пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([0, 1])
calc_logloss(y1, y_pred1)

23.02585088626769

In [8]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [33]:
 X.shape[0], X.shape[1], W.shape

(10, 4, (4,))

In [28]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
#         if i % (iterations / 10) == 0:
    print(i+1, W, err)
    return W

In [29]:
W = eval_model(X_st, y, iterations=100000, eta=1e-5)

100000 [ 0.30086451 -0.7000869   0.66303662  1.28881564] 0.5201517016242287


In [27]:
def calc_pred_proba(W, X):
    z = np.dot(X,W)
    return 1/(1 + np.exp(-z))

In [39]:
calc_pred_proba(W, X_st.T)

array([0.55973568, 0.62564175, 0.76326471, 0.08139663, 0.80654877,
       0.79419326, 0.79975272, 0.19278777, 0.54282496, 0.81943236])

In [40]:
def calc_pred(W, X, treshold = 0.5):
    y_pred = calc_pred_proba(W, X)
    y_pred = np.where(y_pred > treshold, 1, 0 )
    return y_pred

In [50]:
y_pred = calc_pred(W, X_st.T)
y_pred

array([1, 1, 1, 0, 1, 1, 1, 0, 1, 1])

In [53]:
X_st

array([[ 1.        ,  1.        , -0.97958969,  1.        ],
       [ 1.        ,  1.        , -0.56713087,  1.        ],
       [ 1.        ,  2.        , -0.46401617,  2.        ],
       [ 1.        ,  5.        , -0.77336028,  1.        ],
       [ 1.        ,  3.        ,  0.97958969,  2.        ],
       [ 1.        ,  0.        , -0.36090146,  1.        ],
       [ 1.        ,  5.        ,  1.08270439,  3.        ],
       [ 1.        , 10.        ,  2.11385144,  3.        ],
       [ 1.        ,  1.        , -1.08270439,  1.        ],
       [ 1.        ,  2.        ,  0.05155735,  2.        ]])

In [43]:
accuracy = np.sum(y_pred)/y_pred.shape[0] # accuracy
accuracy

0.8

In [44]:
import pandas as pd
conf_matrix = np.array(pd.crosstab(y, y_pred)) # матрица ошибок
conf_matrix

array([[2, 3],
       [0, 5]], dtype=int64)

In [45]:
precition = conf_matrix[1,1]/(conf_matrix[1,1] + conf_matrix[0,1]) # точность 
precition

0.625

In [46]:
recall = conf_matrix[1,1]/(conf_matrix[1,1] + conf_matrix[1,0]) # полнота
recall

1.0

In [49]:
F = 2 * precition * recall / (precition + recall)
F

0.7692307692307693